In [ ]:
year = None
month = None
program = None

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
import scaffolding 
from plots import *

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_ei
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
tq = scaffolding.get_time_query(year, month)

In [ ]:
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)

In [ ]:
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)

In [ ]:
expanded_ct = scaffolding.expand_userinputs(labeled_ct)

In [ ]:
expanded_ct.shape

In [ ]:
expanded_ct = scaffolding.data_quality_check(expanded_ct)
expanded_ct.shape

In [ ]:
## Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

In [ ]:
assert len(expanded_ct[(expanded_ct['Mode_confirm'] == 'Pilot ebike') & (expanded_ct["Replaced_mode"] == "Pilot ebike")]) == 0

In [ ]:
# Energy Impact Calculation
scaffolding.unit_conversions(expanded_ct)

In [ ]:
# group by user
all_trip_user_count = participant_ct_df.groupby("user_id")["user_id"].agg(["count"])
all_trip_user_count

In [ ]:
# group by user
labeled_trip_user_count = expanded_ct.groupby("user_id")["user_id"].agg(["count"])
labeled_trip_user_count

## eBike only

In [ ]:
data_eb = expanded_ct.query("Mode_confirm == 'Pilot ebike'")

In [ ]:
eb_user_count = data_eb.groupby("user_id")["user_id"].count()

In [ ]:
compare_df = pd.concat([eb_user_count, labeled_trip_user_count, all_trip_user_count], axis=1)
compare_df.columns = ["ebike_trips", "labeled_trips", "all_trips"]
compare_df["labeled Trip Pct"] = (compare_df.labeled_trips * 100) / compare_df.all_trips
compare_df["ebike Trip Pct"] = (compare_df.ebike_trips * 100) / compare_df.labeled_trips
compare_df